# Extract and Load Data

## Installing dependencies

In [ ]:
!pip install boto3
!pip install mysql-connector-python

     |████████████████████████████████| 133kB 31.7MB/s 
     |████████████████████████████████| 7.8MB 40.2MB/s 
     |████████████████████████████████| 81kB 12.2MB/s 
     |████████████████████████████████| 143kB 60.3MB/s 
  Created wheel for botocore: filename=botocore-1.20.93-py2.py3-none-any.whl size=7615687 sha256=a1d3cb30f524747e20b7af0a55ac81ee86b50f3e3fe9493972d54950978f20ef
  Stored in directory: /root/.cache/pip/wheels/8f/c8/ec/4e78ad6cfd59266c51515843591b22790eb4754bd644daaaca
Successfully built botocore
ERROR: requests 2.23.0 has requirement urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you'll have urllib3 1.26.5 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
     |████████████████████████████████| 25.4MB 46.3MB/s 


## Importing dependencies

In [ ]:
import json
import boto3
import datetime
import requests
import pandas as pd
from time import sleep
from tabulate import tabulate
from mysql.connector import connect
from configparser import ConfigParser
from botocore.exceptions import NoCredentialsError

## Creating Settings File in FS

In [ ]:
from configparser import ConfigParser
SETTINGS = ConfigParser()

SETTINGS['S3_AUTH'] = {'ACCESS_KEY': '', 'SECRET_KEY': ''}
SETTINGS['RDS_SETTINGS'] = {
    'HOST': '',
    'USER': '',
    'PASSWORD': '',
    'DATABASE': ''
}

# Creating 
with open('settings.ini', 'w') as file:
  SETTINGS.write(file)

# Reading file to memory
SETTINGS.read("settings.ini")

['settings.ini']

## Getting data from external API COVID-19

In [ ]:
url = 'https://api.covid19api.com/countries'
countries = requests.request("GET", url).json()

measurements = []
print('Getting Measurements')
for country in countries:
  if country['Slug'] != 'united-states':
    url = f"https://api.covid19api.com/country/{country['Slug']}?from=2020-01-01T00:00:00Z&to=2021-06-09T00:00:00Z"
    data = requests.request("GET", url).json()
    sleep(1)
    print(f"{country['Slug']} - {len(data)}")
    measurements += data

# Getting US data
week_days = pd.date_range(start='2020-01-01', end='2021-06-09', freq='7d')
for week_day in week_days:
  week_ago = week_day - datetime.timedelta(days=6)
  url = f"https://api.covid19api.com/country/united-states?from={week_ago}&to={week_day}"
  data = requests.request("GET", url).json()
  sleep(1)
  measurements += filter(lambda measurement: measurement['Province'] == "" , data)
  print(f"united-states - {len(data)}")


print('Measurements obtained')

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.5) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


Getting Measurements
angola - 505
luxembourg - 505
american-samoa - 0
mongolia - 505
morocco - 505
rwanda - 505
british-indian-ocean-territory - 0
bosnia-and-herzegovina - 505
costa-rica - 505
kenya - 505
niger - 505
tajikistan - 505
ukraine - 505
wallis-and-futuna-islands - 0
bulgaria - 505
armenia - 505
haiti - 505
monaco - 505
poland - 505
antigua-and-barbuda - 505
cocos-keeling-islands - 0
macedonia - 505
australia - 4537
chad - 505
denmark - 1515
papua-new-guinea - 505
philippines - 505
spain - 505
tanzania - 505
colombia - 505
jamaica - 505
timor-leste - 505
ecuador - 505
russia - 505
réunion - 0
saint-kitts-and-nevis - 505
belgium - 505
norfolk-island - 0
solomon-islands - 505
uzbekistan - 505
antarctica - 0
botswana - 505
british-virgin-islands - 0
chile - 505
united-arab-emirates - 505
estonia - 505
myanmar - 505
south-sudan - 505
zimbabwe - 505
brunei - 505
anguilla - 0
portugal - 505
sri-lanka - 505
grenada - 505
korea-north - 0
cameroon - 505
lithuania - 505
micronesia - 50

## Writing data in measurements.json 

In [ ]:
with open('measurements.json', 'w') as outfile:
    json.dump(measurements, outfile)

## Uploading measurements to AWS S3 

In [ ]:
ACCESS_KEY = SETTINGS["S3_AUTH"]["ACCESS_KEY"]
SECRET_KEY = SETTINGS["S3_AUTH"]["SECRET_KEY"]
S3 = boto3.client('s3', aws_access_key_id=ACCESS_KEY,
                  aws_secret_access_key=SECRET_KEY)


def upload_to_aws(local_file, bucket, s3_file):
    try:
        S3.upload_file(local_file, bucket, s3_file)
        print("Upload Successful")
        print(f'Uploaded file: {local_file}')
        print(f'Upload to bucket: {bucket}')
    except FileNotFoundError:
        print("The file was not found")
    except NoCredentialsError:
        print("Credentials not available")

local_file_name = '/content/measurements.json'
bucket_name = 'ornitorrinco'
s3_file_name = 'measurements.json'
upload_to_aws(local_file_name, bucket_name, s3_file_name)

Upload Successful
Uploaded file: /content/measurements.json
Upload to bucket: ornitorrinco


## Creating tables in AWS RDS - MySQL

In [ ]:
from mysql.connector import connect

mydb = connect(
  host=SETTINGS["RDS_SETTINGS"]["HOST"],
  user=SETTINGS["RDS_SETTINGS"]["USER"],
  password=SETTINGS["RDS_SETTINGS"]["PASSWORD"],
  database=SETTINGS["RDS_SETTINGS"]["DATABASE"]
)
mycursor = mydb.cursor()

# Create table queries

mycursor.execute("""CREATE TABLE IF NOT EXISTS `Locations` (
	`Country` VARCHAR(255) NOT NULL,
	`CountryCode` VARCHAR(255) NOT NULL,
	`Lat` VARCHAR(255) NOT NULL,
	`Lon` VARCHAR(255) NOT NULL,
  `City` VARCHAR(255) NOT NULL,
	`CityCode` VARCHAR(255) NOT NULL,
	`Province` VARCHAR(255) NOT NULL,
	PRIMARY KEY (`Country`)
);""")

mycursor.execute("""CREATE TABLE IF NOT EXISTS `Measurements` (
	`ID` INT NOT NULL AUTO_INCREMENT,
	`Country` VARCHAR(255) NOT NULL,
	`Deaths` INT NOT NULL,
	`Confirmed` INT NOT NULL,
	`Recovered` INT NOT NULL,
	`Active` INT NOT NULL,
	`Date` TIMESTAMP NOT NULL,
  `RegisterId` VARCHAR(255) NOT NULL,
	PRIMARY KEY (`ID`),
  CONSTRAINT `Measurements_fk0` FOREIGN KEY (`Country`)
	REFERENCES `Locations`(`Country`)
);""")

## Migrating data from AWS S3 to AWS RDS

In [ ]:
ACCESS_KEY = SETTINGS["S3_AUTH"]["ACCESS_KEY"]
SECRET_KEY = SETTINGS["S3_AUTH"]["SECRET_KEY"]
S3 = boto3.client('s3', aws_access_key_id=ACCESS_KEY,
                  aws_secret_access_key=SECRET_KEY)

mydb = connect(
  host=SETTINGS["RDS_SETTINGS"]["HOST"],
  user=SETTINGS["RDS_SETTINGS"]["USER"],
  password=SETTINGS["RDS_SETTINGS"]["PASSWORD"],
  database=SETTINGS["RDS_SETTINGS"]["DATABASE"]
)
mycursor = mydb.cursor()

def download_from_aws(local_file, bucket, s3_file):
    try:
        S3.download_file(bucket, s3_file, local_file)
        print("Download Successful")
        print(f'Downloaded file: {s3_file}')
        print(f'Download from bucket: {bucket}')
    except FileNotFoundError:
        print("The file was not found")
    except NoCredentialsError:
        print("Credentials not available")

local_file_name = '/content/s3_measurements.json'
bucket_name = 'ornitorrinco'
file_name = 'measurements.json'
download_from_aws(local_file_name, bucket_name, file_name)

measurements = json.load(open(local_file_name))


queryInsertCountry = f"""INSERT IGNORE INTO Locations 
  (Country, CountryCode, Lat, Lon, City, CityCode, Province) 
  VALUES (%s, %s, %s, %s, %s, %s, %s);"""

queryInsertMeasurement = f"""INSERT INTO Measurements 
  (Country, Deaths, Confirmed, Recovered, Active, Date, RegisterId) 
  VALUES (%s, %s, %s, %s, %s, %s, %s);"""

print('Uploading Data')
locations_data = []
measurements_data = []
for measurement in measurements:
    country = measurement['Country']
    country_code = measurement['CountryCode']
    lat = measurement['Lat']
    lon = measurement['Lon']
    city = measurement['City']
    city_code = measurement['CityCode']
    province = measurement['Province']
    register_id = measurement['ID']
    deaths = measurement['Deaths']
    confirmed = measurement['Confirmed']
    recovered = measurement['Recovered']
    active = measurement['Active']
    date = measurement['Date']

    locations_data.append((country, country_code, lat, lon, city, city_code, province))
    measurements_data.append((country, deaths, confirmed, recovered, active, date, register_id))

    if len(locations_data) == 100:
      mycursor.executemany(queryInsertCountry, locations_data)
      mycursor.executemany(queryInsertMeasurement, measurements_data)
      mydb.commit()
      locations_data = []
      measurements_data = []

mycursor.executemany(queryInsertCountry, locations_data)
mycursor.executemany(queryInsertMeasurement, measurements_data)
mydb.commit()
print("Upload Successful")

Download Successful
Downloaded file: measurements.json
Download from bucket: ornitorrinco
Uploading Data
Upload Successful


## COVID-19 Report

In [ ]:
# Getting Data

from tabulate import tabulate
from mysql.connector import connect

mydb = connect(
  host=SETTINGS["RDS_SETTINGS"]["HOST"],
  user=SETTINGS["RDS_SETTINGS"]["USER"],
  password=SETTINGS["RDS_SETTINGS"]["PASSWORD"],
  database=SETTINGS["RDS_SETTINGS"]["DATABASE"]
)
mycursor = mydb.cursor()

def relate_last_500_days(param, label):
  query = f"""
  SELECT Country, SUM({param}) AS {param}, Date FROM Measurements
  WHERE Country = (
    SELECT Country FROM Measurements
    WHERE Date = '2021-06-09 00:00:00'
    GROUP BY Country, Date
    ORDER BY Date DESC, SUM({param}) DESC
    LIMIT 1
  )
  GROUP BY Country, Date
  ORDER BY Date DESC LIMIT 500;
  """
  mycursor.execute(query)
  data = mycursor.fetchall()

  country = data[0][0]
  print(f'\nMost {label}: {country}')
  print(tabulate([(str(cases), date.strftime('%d/%m/%Y')) for (_, cases, date) in data], headers=[param, 'Date']))

def relate_top_10(param, label):
  query = f"""
  SELECT Country, SUM({param}) AS {param} FROM Measurements
  WHERE Date = '2021-06-09 00:00:00'
  GROUP BY Country, Date
  ORDER BY Date DESC, SUM({param}) DESC
  LIMIT 10;
  """
  mycursor.execute(query)
  data = mycursor.fetchall()

  print(f'\nTop 10 Countries with Most {label}')
  print(tabulate([(country, str(cases)) for (country, cases) in data], headers=['Country', param]))

### Daily Report of Country with Most Confirmed Cases

In [ ]:
relate_last_500_days('Confirmed', 'Confirmed Cases')


Most Confirmed Cases: United States of America
  Confirmed  Date
-----------  ----------
   33409753  09/06/2021
   33391107  08/06/2021
   33378096  07/06/2021
   33362600  06/06/2021
   33357205  05/06/2021
   33343297  04/06/2021
   33326437  03/06/2021
   33307363  02/06/2021
   33290450  01/06/2021
   33267507  31/05/2021
   33261731  30/05/2021
   33254998  29/05/2021
   33242999  28/05/2021
   33221141  27/05/2021
   33193687  26/05/2021
   33169715  25/05/2021
   33146976  24/05/2021
   33121157  23/05/2021
   33108291  22/05/2021
   33088492  21/05/2021
   33060540  20/05/2021
   33030332  19/05/2021
   33001039  18/05/2021
   32973249  17/05/2021
   32944627  16/05/2021
   32927740  15/05/2021
   32898886  14/05/2021
   32856627  13/05/2021
   32818551  12/05/2021
   32782712  11/05/2021
   32749068  10/05/2021
   32712264  09/05/2021
   32690834  08/05/2021
   32657158  07/05/2021
   32609021  06/05/2021
   32561469  05/05/2021
   32516756  04/05/2021
   32476040  03/05/202

### Daily Report of Country with Most Deaths

In [ ]:
relate_last_500_days('Deaths', 'Deaths')


Most Deaths: United States of America
  Deaths  Date
--------  ----------
  598316  09/06/2021
  597880  08/06/2021
  597504  07/06/2021
  597194  06/06/2021
  596943  05/06/2021
  596573  04/06/2021
  596061  03/06/2021
  595451  02/06/2021
  594902  01/06/2021
  594264  31/05/2021
  594122  30/05/2021
  593996  29/05/2021
  593669  28/05/2021
  593117  27/05/2021
  591772  26/05/2021
  590802  25/05/2021
  590179  24/05/2021
  589742  23/05/2021
  589551  22/05/2021
  589068  21/05/2021
  588456  20/05/2021
  587780  19/05/2021
  587116  18/05/2021
  586349  17/05/2021
  585951  16/05/2021
  585677  15/05/2021
  585187  14/05/2021
  584539  13/05/2021
  583779  12/05/2021
  583013  11/05/2021
  582329  10/05/2021
  581917  09/05/2021
  581672  08/05/2021
  581045  07/05/2021
  580289  06/05/2021
  579490  05/05/2021
  578697  04/05/2021
  577823  03/05/2021
  577322  02/05/2021
  576987  01/05/2021
  576273  30/04/2021
  575522  29/04/2021
  574650  28/04/2021
  573684  27/04/2021
 

### Top 10 Countries with Most Confirmed Cases

In [ ]:
relate_top_10('Confirmed', 'Confirmed Cases')


Top 10 Countries with Most Confirmed Cases
Country                     Confirmed
------------------------  -----------
United States of America     33409753
India                        29182532
Brazil                       17122877
France                        5787125
Turkey                        5306690
Russian Federation            5096657
United Kingdom                4551694
Italy                         4237790
Argentina                     4038528
Germany                       3715870


### Top 10 Countries with Most Deaths

In [ ]:
relate_top_10('Deaths', 'Deaths')


Top 10 Countries with Most Deaths
Country                     Deaths
------------------------  --------
United States of America    598316
Brazil                      479515
India                       355705
Mexico                      229353
Peru                        187157
United Kingdom              128124
Italy                       126767
Russian Federation          122802
France                      110364
Colombia                     93394
